In [1]:
from preprocess import *
from COLD import * 
from LBP import * 
from HOG import *
import pandas as pd

In [2]:
mainDataset=pd.read_csv('NonNormalized.csv')
featuresData=mainDataset.iloc[:,3:]
mean = featuresData.mean()
std = featuresData.std()

def predict(imgName):
    linesTable = []
    linesList = PreprocessImage(imgName,isPrediction=True)
    for line in linesList:
        coldHist = ColdFeature(line)
        lbpHist = extract_lbp(line)
        hogHist = calculateHOG(line)
        dataValues = [imgName]
        dataValues.extend(coldHist)
        dataValues.extend(lbpHist)
        dataValues.extend(hogHist)
        linesTable.append(dataValues)

    COLDLabels = ['COLD_'+str(i) for i in range(100)]
    LBPLabels = ['LBP_'+str(i) for i in range(256)]
    HOGLabels = ['HOG_'+str(i) for i in range(144)]
    labels = ['ImgName']
    labels.extend(COLDLabels)
    labels.extend(LBPLabels)
    labels.extend(HOGLabels)

    linesTable = np.array(linesTable)
    df = pd.DataFrame(data=linesTable,columns=labels)

    for column in df: 
        if column != 'Gender' and column != 'ImgName':
            df[column] = df[column].astype(float)

    #remove columns not found in the normal dataset
    for col in df:
        if not col in mainDataset.columns:
            df=df.drop(columns=[col])

    #Calculate standard normalization
    df.iloc[:,1:] = df.iloc[:,1:] - mean / std


    df.to_csv('test.csv')
    #Classify here


In [6]:
predict('M229.jpg')